# Analyze EIBP Log Results

## Input Required Information

| Variable | Use |
| --- | --- |
| LOG_DIR_PATH | Location of the log directory. | 

In [2]:
LOG_DIR_PATH = "/home/fabric/work/FABRIC-EIBP/logs"

Convergence time: time taken from 1 failure node to the last node heard about it.

Control Overheard: Size of the failure message overall

Churn rate : how far the error message propagates. 

IF_DIRECT_DOWN: nodes detect interface is down

IF_TIMER_DOWN:dead timer is down or expiring.

# Convergence Time

In [3]:
import re
import os

# Regular expression patterns to match T1 and T2
pattern_T1 = re.compile(r'^IF_DIRECT_DOWN:(\d+\.\d+)$')
pattern_T2 = re.compile(r'^DELETE MESSAGE RECEIVED: CURRENT_TIME:(\d+\.\d+)$')

# Initialize variables for T1 and T2
T1 = None
T2 = float('-inf')

# Iterate through each log file
for filename in os.listdir(LOG_DIR_PATH):
    if filename.endswith('.log'):
        with open(os.path.join(LOG_DIR_PATH, filename), 'r') as file:
            # Read each line in the log file
            for line in file:
                # Match pattern for T1 (IF_DIRECT_DOWN)
                match_T1 = pattern_T1.match(line.strip())
                if match_T1:
                    # Capture T1 (time of IF_DIRECT_DOWN)
                    T1 = float(match_T1.group(1))
                
                # Match pattern for T2 (LAST DELETE MESSAGE CURRENT_TIME)
                match_T2 = pattern_T2.match(line.strip())
                if match_T2:
                    # Capture the current time for DELETE MESSAGE RECEIVED
                    current_time = float(match_T2.group(1))
                    # Update T2 with the latest current_time
                    if current_time > T2:
                        T2 = current_time

# Calculate and display the difference convergence time (T2 - T1) in seconds
if T1 is not None:
    result = T2 - T1
    print(f"Convergence time: {result} seconds")
else:
    print("IF_DIRECT_DOWN not found in the logs.")


Convergence time: 3.2277841567993164 seconds


# Control Overhead

In [4]:
import os

eth_sizes = []

# Read the log files in the specified directory
for filename in os.listdir(LOG_DIR_PATH):
    if filename.endswith('.log'):
        file_path = os.path.join(LOG_DIR_PATH, filename)
        found_delete_message = False  # Reset for each file
        capture_eth_size = False  # Flag to capture ETH_SIZE values
        
        # Read each log file
        with open(file_path, 'r') as file:
            log_lines = file.readlines()
            
            # Parse the log to extract ETH_SIZE values under DELETE MESSAGE RECEIVED: CURRENT_TIME
            for line in log_lines:
                if "DELETE MESSAGE RECEIVED" in line:
                    found_delete_message = True
                  #  print(f"DELETE MESSAGE RECEIVED print: {line}")
                    capture_eth_size = True  # Set flag to capture ETH_SIZE values after this line
                elif capture_eth_size and "ETH_SIZE" in line:
                  #  print(f"eth_size: {line}")
                    eth_size = line.split(":")[1].strip()
                    eth_sizes.append(int(eth_size))
                elif capture_eth_size and "DELETE MESSAGE RECEIVED" in line:
                    capture_eth_size = False  # Reset the flag if another DELETE MESSAGE line is encountered
                
# Calculate the sum of ETH_SIZE values
sum_eth_sizes = sum(eth_sizes)
print(f"Control Overhead:{sum_eth_sizes} bytes")


Control Overhead:196 bytes


# CHURN PERCENTAGE

In [5]:
import os

churn_count = 0
total_files = 0

# Loop through all files in the current directory
for filename in os.listdir(LOG_DIR_PATH):
    if filename.endswith('.log'):
        total_files += 1
        # Open the file and check for churn
        with open(os.path.join(LOG_DIR_PATH, filename), 'r') as file:
            churn_found = any('CHURN_TRUE' in line.strip() for line in file)
            if churn_found:
                churn_count += 1

# Calculate churn percentage
if total_files > 0:
    churn_percent = (churn_count / total_files) * 100
else:
    churn_percent = 0

# Display the churn percentage
print(f"CHURN PERCENTAGE: {churn_percent:.2f}%")


CHURN PERCENTAGE: 100.00%
